In [51]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np 
import pandas as pd
import datetime as dt

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

import gc

from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans


import folium
import geopandas as gpd
from haversine import haversine

import optuna
import xgboost as xgb

from sklearn.ensemble import RandomForestRegressor

pd.set_option('display.max_columns', None)


bold = ['\033[1m', '\033[0m']
seed = 228

In [52]:
train = pd.read_csv('train.csv') # index_col=0 is the id column
test = pd.read_csv('test.csv') # index_col=0 is the id column
sample_submission = pd.read_csv('sample_submission.csv')
y = train['emission']

## 1. ID Generation:
The code first generates a unique ID for each row by combining latitude and longitude. It extracts digits from both coordinates, concatenates them, and then maps them to a new unique ID.

In [53]:
def get_id(row):
    return int(''.join(filter(str.isdigit, str(row['latitude']))) + ''.join(filter(str.isdigit, str(row['longitude']))))

train['id'] = train[['latitude', 'longitude']].apply(lambda row: get_id(row), axis=1)
test['id'] = test[['latitude', 'longitude']].apply(lambda row: get_id(row), axis=1)
new_ids = {id_: new_id for new_id, id_ in enumerate(train['id'].unique())}
train['id'] = train['id'].map(new_ids)
test['id'] = test['id'].map(new_ids)

## 2. Extract Month from Year and Week:
A function to extract the month given a year and week number is defined.

In [54]:
def get_month(row):
    date = dt.datetime.strptime(f'{row["year"]}-{row["week_no"]+1}-1', "%Y-%W-%w")
    return date.month


## 3. Define Geographic Points of Interest:
Several important geographic locations are defined for later distance computations.

In [55]:
rwanda_center = (-1.9607, 29.9707)
park_biega = (-1.8866, 28.4518)
kirumba = (-0.5658, 29.1714)
massif = (-3.42, 28.592)
lake = (-2.0073, 31.6269)

## 4. Compute Distances to Cluster Centers:
The code computes distances from given points to several cluster centers.

In [56]:
def cluster_features(df, cluster_centers):
    for i, cc in enumerate(cluster_centers.values()):
        df[f'cluster_{i}'] = df.apply(lambda x: haversine((x['latitude'], x['longitude']), cc, unit='ft'), axis=1)
    return df

## 5. Data Preprocessing:
This section is the bulk of the code. It involves:

 1. Filtering columns
 2. Forward and backward filling missing values
 2. Creating several lag and rotational features
 4. Calculating distances to defined geographic locations
 5. Extracting month and identifying periods of COVID-19 and lockdowns

In [57]:
def preprocessing(df):
    # Select important columns
   
    cols_save = ['id', 'latitude', 'longitude', 'year', 'week_no', 'Ozone_solar_azimuth_angle']
    df = df[cols_save]
    
    # Fill missing values for 'Ozone_solar_azimuth_angle'
    good_col = 'Ozone_solar_azimuth_angle'
    df[good_col] = df.groupby(['id', 'year'])[good_col].ffill().bfill()
    df[f'{good_col}_lag_1'] = df.groupby(['id', 'year'])[good_col].shift(1).fillna(0)
    
    # Compute rotated features for latitude and longitude
    df['rot_15_x'] = (np.cos(np.radians(15)) * df['longitude']) + \
                     (np.sin(np.radians(15)) * df['latitude'])
    
    df['rot_15_y'] = (np.cos(np.radians(15)) * df['latitude']) + \
                     (np.sin(np.radians(15)) * df['longitude'])

    df['rot_30_x'] = (np.cos(np.radians(30)) * df['longitude']) + \
                     (np.sin(np.radians(30)) * df['latitude'])

    df['rot_30_y'] = (np.cos(np.radians(30)) * df['latitude']) + \
                     (np.sin(np.radians(30)) * df['longitude'])
    
    # Calculate distances to specified geographic locations
    for col, coors in zip(
        ['dist_rwanda', 'dist_park', 'dist_kirumba', 'dist_massif', 'dist_lake'], 
        [rwanda_center, park_biega, kirumba, massif, lake]
    ):
        df[col] = df.apply(lambda x: haversine((x['latitude'], x['longitude']), coors, unit='ft'), axis=1)
    
    # Extract month and label periods of COVID-19 and lockdowns
    df['month'] = df[['year', 'week_no']].apply(lambda row: get_month(row), axis=1)
    df['is_covid'] = (df['year'] == 2020) & (df['month'] > 2) | (df['year'] == 2021) & (df['month'] == 1)
    df['is_lockdown'] = (df['year'] == 2020) & ((df['month'].isin([3,4])))
    
    df.fillna(0, inplace=True)
    return df

train = preprocessing(train)
test = preprocessing(test)


In [58]:
train.head()

,id,latitude,longitude,year,week_no,Ozone_solar_azimuth_angle,Ozone_solar_azimuth_angle_lag_1,rot_15_x,rot_15_y,rot_30_x,rot_30_y,dist_rwanda,dist_park,dist_kirumba,dist_massif,dist_lake,month,is_covid,is_lockdown
0,0,-0.51,29.29,2019,0,-138.786446,0.000000,28.15997,7.088188,25.110884,14.203327,584571.8739,587934.30305,47814.701579,1.091677e+06,1.012320e+06,1,False,False
1,0,-0.51,29.29,2019,1,-143.097868,-138.786446,28.15997,7.088188,25.110884,14.203327,584571.8739,587934.30305,47814.701579,1.091677e+06,1.012320e+06,1,False,False
2,0,-0.51,29.29,2019,2,-135.364627,-143.097868,28.15997,7.088188,25.110884,14.203327,584571.8739,587934.30305,47814.701579,1.091677e+06,1.012320e+06,1,False,False
3,0,-0.51,29.29,2019,3,-137.489602,-135.364627,28.15997,7.088188,25.110884,14.203327,584571.8739,587934.30305,47814.701579,1.091677e+06,1.012320e+06,1,False,False
4,0,-0.51,29.29,2019,4,-136.448518,-137.489602,28.15997,7.088188,25.110884,14.203327,584571.8739,587934.30305,47814.701579,1.091677e+06,1.012320e+06,2,False,False


In [60]:
test.head()

,id,latitude,longitude,year,week_no,Ozone_solar_azimuth_angle,Ozone_solar_azimuth_angle_lag_1,rot_15_x,rot_15_y,rot_30_x,rot_30_y,dist_rwanda,dist_park,dist_kirumba,dist_massif,dist_lake,month,is_covid,is_lockdown
0,0,-0.51,29.29,2022,0,-133.047562,0.000000,28.15997,7.088188,25.110884,14.203327,584571.8739,587934.30305,47814.701579,1.091677e+06,1.012320e+06,1,False,False
1,0,-0.51,29.29,2022,1,-139.171039,-133.047562,28.15997,7.088188,25.110884,14.203327,584571.8739,587934.30305,47814.701579,1.091677e+06,1.012320e+06,1,False,False
2,0,-0.51,29.29,2022,2,-142.989044,-139.171039,28.15997,7.088188,25.110884,14.203327,584571.8739,587934.30305,47814.701579,1.091677e+06,1.012320e+06,1,False,False
3,0,-0.51,29.29,2022,3,-135.011886,-142.989044,28.15997,7.088188,25.110884,14.203327,584571.8739,587934.30305,47814.701579,1.091677e+06,1.012320e+06,1,False,False
4,0,-0.51,29.29,2022,4,-135.499142,-135.011886,28.15997,7.088188,25.110884,14.203327,584571.8739,587934.30305,47814.701579,1.091677e+06,1.012320e+06,1,False,False


## 6. Clustering:
Based on latitude and longitude, KMeans clustering is applied to group locations into 12 clusters

In [61]:
df = pd.concat([train, test], axis=0, ignore_index=True)
coordinates = df[['latitude', 'longitude']].values
clustering = KMeans(n_clusters=25, max_iter=100, random_state=seed).fit(coordinates)
cluster_centers = {i: tuple(centroid) for i, centroid in enumerate(clustering.cluster_centers_)}
df = cluster_features(df, cluster_centers)

## 7. Splitting and Cleaning:
Finally, the preprocessed dataframe is split back into train and test, and unused variables are removed.

In [62]:

train = df.iloc[:-len(test),:]
test = df.iloc[-len(test):,:]
del df

X = train.drop('id', axis=1)
test = test.drop('id', axis=1)
test=test.reset_index(drop=True)

In [41]:
import optuna
import xgboost as xgb
from optuna.integration import XGBoostPruningCallback

dtrain = xgb.DMatrix(X, label=y)

def objective(trial):
    param = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        'lambda': trial.suggest_float('lambda', 1e-5, 1, log=True),
        'alpha': trial.suggest_float('alpha', 1e-5, 1, log=True),
        'subsample': trial.suggest_float('subsample', 0.1, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 9),
        'eta': trial.suggest_float('eta', 1e-5, 1, log=True),
        'gamma': trial.suggest_float('gamma', 0, 0.4)

    }
    
    pruning_callback = XGBoostPruningCallback(trial, 'test-rmse')
    cv_result = xgb.cv(param, dtrain, num_boost_round=50, nfold=3, callbacks=[pruning_callback], verbose_eval=False)
    
    return cv_result["test-rmse-mean"].values[-1]

study = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=30, show_progress_bar=True)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2023-08-06 14:54:36,081] A new study created in memory with name: no-name-b35a888e-b312-4e4a-9f62-369a359019f8


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2023-08-06 14:54:55,354] Trial 0 finished with value: 16.151111685217703 and parameters: {'booster': 'dart', 'lambda': 0.6243022272554737, 'alpha': 1.5357131825482986e-05, 'subsample': 0.6440796347561257, 'colsample_bytree': 0.8675713150082035, 'max_depth': 9, 'min_child_weight': 1, 'eta': 0.3005323563211222, 'gamma': 0.1527109982607764}. Best is trial 0 with value: 16.151111685217703.
[I 2023-08-06 14:55:09,265] Trial 1 finished with value: 157.600963129494 and parameters: {'booster': 'dart', 'lambda': 8.395688030423453e-05, 'alpha': 0.0002724392400887872, 'subsample': 0.6415145757684722, 'colsample_bytree': 0.8158171917951432, 'max_depth': 6, 'min_child_weight': 5, 'eta': 0.0011657342867026686, 'gamma': 0.1830503455545267}. Best is trial 0 with value: 16.151111685217703.
[I 2023-08-06 14:55:28,299] Trial 2 finished with value: 164.1665550146565 and parameters: {'booster': 'dart', 'lambda': 0.00040787644435426724, 'alpha': 0.00035126531143328196, 'subsample': 0.784576948929023, 'co

In [ ]:
#   Value:  15.28230117578614
#   Params: 
#     booster: dart
#     lambda: 0.0010459589924578867
#     alpha: 0.0005372374922859215
#     subsample: 0.4057031970586893
#     colsample_bytree: 0.9047262730204358
#     max_depth: 9
#     min_child_weight: 2
#     eta: 0.2955965909548593
#     gamma: 0.1381495694268014

In [64]:
rf = RandomForestRegressor(random_state=seed, n_jobs=-1)
rf.fit(X, y)

sample_submission['emission'] = rf.predict(test)
sample_submission.to_csv('submission.csv', index=False)
sample_submission

,ID_LAT_LON_YEAR_WEEK,emission
0,ID_-0.510_29.290_2022_00,4.024762
1,ID_-0.510_29.290_2022_01,4.117443
2,ID_-0.510_29.290_2022_02,4.058612
3,ID_-0.510_29.290_2022_03,4.163972
4,ID_-0.510_29.290_2022_04,4.182518
...,...,...
24348,ID_-3.299_30.301_2022_44,29.554739
24349,ID_-3.299_30.301_2022_45,29.705708
24350,ID_-3.299_30.301_2022_46,29.951771
24351,ID_-3.299_30.301_2022_47,29.806629
